https://www.tensorflow.org/tutorials/layers

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

In [2]:
def cnn_model_fn(features, labels, mode):
    """Model function for CNN"""
    
    input_layer = tf.reshape(features['x'], [-1, 28, 28, 1])
    
    conv1 = tf.layers.conv2d(
        inputs=input_layer,
        filters=32,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu
    )
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
    
    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=64,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu
    )
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
    dense = tf.layers.dense(inputs=pool2_flat,
                           units=2014,
                           activation=tf.nn.relu)
    
    dropout = tf.layers.dropout(
        inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN
    )
    
    logits = tf.layers.dense(inputs=dropout, units=10)
    
    predictions = {
        "classes": tf.argmax(input=logits, axis=1),
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor"),
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
    loss = tf.losses.softmax_cross_entropy(
        onehot_labels=onehot_labels, logits=logits
    )
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step()
        )
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
    
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(
        labels=labels, predictions=predictions["classes"])
    }
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)


In [3]:
mnist = tf.contrib.learn.datasets.load_dataset("mnist")
train_data = mnist.train.images
train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
eval_data = mnist.test.images
eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

Extracting MNIST-data\train-images-idx3-ubyte.gz
Extracting MNIST-data\train-labels-idx1-ubyte.gz
Extracting MNIST-data\t10k-images-idx3-ubyte.gz
Extracting MNIST-data\t10k-labels-idx1-ubyte.gz


In [4]:
print(train_data.shape)
print(train_labels.shape)

(55000, 784)
(55000,)


In [5]:
mnist_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn, model_dir="E:\\temp")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_log_step_count_steps': 100, '_service': None, '_tf_random_seed': None, '_num_ps_replicas': 0, '_save_checkpoints_secs': 600, '_num_worker_replicas': 1, '_task_type': 'worker', '_save_checkpoints_steps': None, '_model_dir': 'E:\\temp', '_task_id': 0, '_save_summary_steps': 100, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000023CD46774E0>, '_session_config': None, '_is_chief': True, '_keep_checkpoint_max': 5, '_master': '', '_keep_checkpoint_every_n_hours': 10000}


In [6]:
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=50)

In [7]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
    batch_size=100,
    num_epochs=None,
    shuffle=True
)

mnist_classifier.train(
    input_fn=train_input_fn,
    steps=20000,
    #hooks=[logging_hook]
)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into E:\temp\model.ckpt.
INFO:tensorflow:loss = 2.30733, step = 1
INFO:tensorflow:global_step/sec: 118.728
INFO:tensorflow:loss = 2.29815, step = 101 (0.842 sec)
INFO:tensorflow:global_step/sec: 130.705
INFO:tensorflow:loss = 2.27955, step = 201 (0.766 sec)
INFO:tensorflow:global_step/sec: 129.183
INFO:tensorflow:loss = 2.27678, step = 301 (0.773 sec)
INFO:tensorflow:global_step/sec: 130.201
INFO:tensorflow:loss = 2.27106, step = 401 (0.768 sec)
INFO:tensorflow:global_step/sec: 129.862
INFO:tensorflow:loss = 2.2544, step = 501 (0.771 sec)
INFO:tensorflow:global_step/sec: 128.855
INFO:tensorflow:loss = 2.23111, step = 601 (0.775 sec)
INFO:tensorflow:global_step/sec: 130.542
INFO:tensorflow:loss = 2.21025, step = 701 (0.766 sec)
INFO:tensorflow:global_step/sec: 130.542
INFO:tensorflow:loss = 2.18121, step = 801 (0.766 sec)
INFO:tensorflow:global_step/sec: 129.856
INFO:tensorflow:loss = 2.1725, step = 901

INFO:tensorflow:global_step/sec: 131.402
INFO:tensorflow:loss = 0.212588, step = 8401 (0.761 sec)
INFO:tensorflow:global_step/sec: 130.713
INFO:tensorflow:loss = 0.1724, step = 8501 (0.766 sec)
INFO:tensorflow:global_step/sec: 131.402
INFO:tensorflow:loss = 0.237961, step = 8601 (0.762 sec)
INFO:tensorflow:global_step/sec: 129.525
INFO:tensorflow:loss = 0.22733, step = 8701 (0.772 sec)
INFO:tensorflow:global_step/sec: 131.575
INFO:tensorflow:loss = 0.213552, step = 8801 (0.760 sec)
INFO:tensorflow:global_step/sec: 128.358
INFO:tensorflow:loss = 0.21458, step = 8901 (0.778 sec)
INFO:tensorflow:global_step/sec: 130.201
INFO:tensorflow:loss = 0.141661, step = 9001 (0.767 sec)
INFO:tensorflow:global_step/sec: 130.859
INFO:tensorflow:loss = 0.181952, step = 9101 (0.764 sec)
INFO:tensorflow:global_step/sec: 130.2
INFO:tensorflow:loss = 0.170217, step = 9201 (0.768 sec)
INFO:tensorflow:global_step/sec: 129.357
INFO:tensorflow:loss = 0.185642, step = 9301 (0.775 sec)
INFO:tensorflow:global_ste

INFO:tensorflow:global_step/sec: 128.193
INFO:tensorflow:loss = 0.131907, step = 16701 (0.782 sec)
INFO:tensorflow:global_step/sec: 131.229
INFO:tensorflow:loss = 0.109103, step = 16801 (0.760 sec)
INFO:tensorflow:global_step/sec: 128.358
INFO:tensorflow:loss = 0.195043, step = 16901 (0.781 sec)
INFO:tensorflow:global_step/sec: 129.357
INFO:tensorflow:loss = 0.116751, step = 17001 (0.773 sec)
INFO:tensorflow:global_step/sec: 129.492
INFO:tensorflow:loss = 0.107641, step = 17101 (0.770 sec)
INFO:tensorflow:global_step/sec: 128.689
INFO:tensorflow:loss = 0.0670252, step = 17201 (0.778 sec)
INFO:tensorflow:global_step/sec: 129.693
INFO:tensorflow:loss = 0.0441402, step = 17301 (0.771 sec)
INFO:tensorflow:global_step/sec: 131.229
INFO:tensorflow:loss = 0.142498, step = 17401 (0.762 sec)
INFO:tensorflow:global_step/sec: 129.357
INFO:tensorflow:loss = 0.0895851, step = 17501 (0.773 sec)
INFO:tensorflow:global_step/sec: 130.031
INFO:tensorflow:loss = 0.184601, step = 17601 (0.768 sec)
INFO:te

In [8]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data},
    y=eval_labels,
    num_epochs=1,
    shuffle=False)
eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

INFO:tensorflow:Starting evaluation at 2018-01-14-04:28:18
INFO:tensorflow:Restoring parameters from E:\temp\model.ckpt-20000
INFO:tensorflow:Finished evaluation at 2018-01-14-04:28:18
INFO:tensorflow:Saving dict for global step 20000: accuracy = 0.9705, global_step = 20000, loss = 0.0989317
{'global_step': 20000, 'loss': 0.098931745, 'accuracy': 0.97049999}
